In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import numpy as np

from helper_functs import *
from basic_players import *

In [105]:
class Game:
	def __init__(self, players, dice_count, verbose=False, verbose_bids=False) -> None:
		# setting up a rng generator
		self.rng = np.random.default_rng()

		self.verbose = verbose
		self.verbose_bids = verbose_bids

		self.out_log = []

		# storing the players and the dice numbers, and shuffling the player orders
		self.game_overview = {"players": self.rng.permutation(players), "dice_count": [dice_count]*len(players)}

		self.total_players = len(players)
		self.alive_players = len(players)
		self.next_player = 0
		self.round_log = {"player": [], "dice_count": [], "amount": [], "face": []}
		self.round_data = {"total_dice": 0, "total_counts": [0]*6, "player_dice": []}

		self.init_round()

	
	def init_round(self):
		self.round_log = {"player": [], "dice_count": [], "amount": [], "face": []}
		self.round_data = {"total_dice": 0, "total_counts": [0]*6, "player_dice": []}

		if self.verbose:
			print("initialising round:")

		print("")
		for player in range(len(self.game_overview["players"])):
			player_dice = [0]*6
			# if self.game_overview["dice_count"][player] == 0:
			# 	self.round_data["player_dice"].append(player_dice)
			# 	continue

			for dice in range(self.game_overview["dice_count"][player]):
				value = self.rng.integers(1,7)
				self.round_data["total_dice"]+=1
				self.round_data["total_counts"][value-1] += 1
				player_dice[value-1] += 1
			# print(player_dice)
			self.round_data["player_dice"].append(player_dice)
			print(f"- {np.sum(player_dice)}: {player_dice}")
	
	def play_round(self):
		while True:
			if self.game_overview["dice_count"][self.next_player] == 0: # the player has no dice, so they are skipped
				self.next_player += 1
				self.next_player %= self.total_players
				continue

			amount, face = self.game_overview["players"][self.next_player](np.sum(self.round_data["total_dice"]), self.round_data["player_dice"][self.next_player], self.round_log)

			if self.verbose_bids:
				if amount != 0 and face != 0:
					print(f"player {self.next_player} has bid {amount} {face}'s")
				else: # calling
					print(f"player {self.next_player} is calling out player {self.round_log["player"][-1]}'s bid of {self.round_log["amount"][-1]} {self.round_log["face"][-1]}'s")

			
			if amount > 2*self.round_data["total_dice"]:
				print("while loop is going infinite")
				return
				break

			# handling calling
			if amount == 0 and face == 0:
				# evaluate the previous bid
				final_counts = self.round_data["total_counts"]
				for i in range(1,6): # adding the ones to all other values
					final_counts[i] += final_counts[0]
				prev_amount = self.round_log["amount"][-1]
				prev_face = self.round_log["face"][-1]
				if prev_amount > final_counts[prev_face-1]:
					# caller is right

					# decreasing previous player's dice amount
					prev_player = self.round_log["player"][-1]
					self.game_overview["dice_count"][prev_player] -= 1
					if self.verbose:
						print(f"player {prev_player} lost a dice!")
					if self.game_overview["dice_count"][prev_player] <= 0:
						print(f"\nPlayer {prev_player} finished in {self.alive_players}'th place\n")
						self.out_log.append(prev_player)
						self.alive_players -= 1
					if self.alive_players == 1:
						# a player has won the game
						for player in range(len(self.game_overview["players"])):
							if self.game_overview["dice_count"][player] != 0:
								print(f"\nPlayer {player} has won!")
								self.out_log.append(player)
								print(f"\nresults:")
								for i in range(self.total_players):
									print(f"{self.total_players - i}: player {self.out_log[i]}")
								return
					self.init_round()
					self.next_player += 1
					self.next_player %= self.total_players
					return
				else:
					# caller is wrong
					self.game_overview["dice_count"][self.next_player] -= 1
					if self.verbose:
						print(f"player {self.next_player} lost a dice!")
					if self.game_overview["dice_count"][self.next_player] == 0:
						print(f"\nPlayer {self.next_player} finished in {self.alive_players}'th place\n")
						self.out_log.append(self.next_player)
						self.alive_players -= 1
					if self.alive_players == 1:
						# a player has won the game
						for player in range(len(self.game_overview["players"])):
							if self.game_overview["dice_count"][player] != 0:
								print(f"\nPlayer {player} has won!")
								self.out_log.append(player)
								print(f"\nresults:")
								for i in range(self.total_players):
									print(f"{self.total_players - i}: player {self.out_log[i]}")
								return
					self.init_round()
					self.next_player += 1
					self.next_player %= self.total_players
					return


			if len(self.round_log["player"]) != 0:
				if not is_valid(self.round_log["face"][-1], self.round_log["amount"][-1], face, amount):
					print(f"player number {self.next_player} submitted an invalid bid!")
					#! TODO handle invalid bids
					break
			else:
				if face not in [2,3,4,5,6] or amount < 1:
					print(f"player number {self.next_player} submitted an invalid bid!")
					#! TODO handle invalid bids
					break


			self.round_log["face"].append(face)
			self.round_log["amount"].append(amount)
			self.round_log["player"].append(self.next_player)
			self.round_log["dice_count"].append(self.game_overview["dice_count"][self.next_player])

			self.next_player += 1
			self.next_player %= self.total_players
			

	def play_rounds(self, rounds=0):
		if rounds == 0:
			# run until finished
			while self.alive_players > 1:
				self.play_round()
			pass
		else:
			for round in range(rounds):
				if self.alive_players == 1:
					return
				self.play_round()


In [106]:
# players = [incrementer2, incrementer, ExpectedPlayer, ExpectedPlayer, ExpectedPlayer2, ExpectedPlayer2]
players = [incrementer2, ExpectedPlayer2, ExpectedPlayer]

In [107]:
game = Game(players, 5, verbose=True, verbose_bids=True)
game.game_overview

initialising round:

- 5: [3, 1, 0, 1, 0, 0]
- 5: [0, 0, 0, 3, 0, 2]
- 5: [1, 1, 1, 1, 1, 0]


{'players': array([<function incrementer2 at 0x7f93492ba3e0>,
        <function ExpectedPlayer2 at 0x7f93492bb7e0>,
        <function ExpectedPlayer at 0x7f93492bb9c0>], dtype=object),
 'dice_count': [5, 5, 5]}

In [108]:
game.play_rounds()

player 0 has bid 1 2's
player 1 has bid 6 4's
player 2 is calling out player 1's bid of 6 4's
player 2 lost a dice!
initialising round:

- 5: [0, 2, 1, 0, 1, 1]
- 5: [1, 0, 0, 2, 2, 0]
- 4: [1, 2, 0, 0, 0, 1]
player 0 has bid 1 2's
player 1 has bid 5 5's
player 2 is calling out player 1's bid of 5 5's
player 2 lost a dice!
initialising round:

- 5: [1, 1, 1, 0, 1, 1]
- 5: [1, 0, 0, 2, 0, 2]
- 3: [1, 1, 0, 0, 0, 1]
player 0 has bid 1 2's
player 1 has bid 5 6's
player 2 is calling out player 1's bid of 5 6's
player 2 lost a dice!
initialising round:

- 5: [1, 2, 1, 0, 1, 0]
- 5: [1, 0, 0, 1, 2, 1]
- 2: [0, 1, 0, 1, 0, 0]
player 0 has bid 1 2's
player 1 has bid 4 5's
player 2 is calling out player 1's bid of 4 5's
player 2 lost a dice!
initialising round:

- 5: [1, 0, 1, 1, 1, 1]
- 5: [1, 1, 1, 0, 2, 0]
- 1: [0, 0, 1, 0, 0, 0]
player 0 has bid 1 2's
player 1 has bid 4 5's
player 2 is calling out player 1's bid of 4 5's
player 2 lost a dice!

Player 2 finished in 3'th place

initialising r